<a href="https://colab.research.google.com/github/eminshall/Notebooks-Youtube-Channel/blob/main/Finding_the_Efficient_Frontier_with_Monte_Carlo_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

In [ ]:
n_portfolios = 10 ** 5
n_days = 252
assets = []
assets.sort()
n_assets = len(assets)

start_date = '2024-01-01'
end_date = '2024-12-31'

In [ ]:
df = yf.download(assets,
                 start = start_date,
                 end = end_date)['Close']

returns = df.pct_change().dropna()
avg_returns = returns.mean() * n_days
cov_matrix = returns.cov() * n_days

In [ ]:
np.random.seed(42)
weights = np.random.random(size=(n_portfolios, n_assets))
weights /= np.sum(weights, axis=1)[:, np.newaxis]

In [ ]:
portf_returns = np.dot(weights, avg_returns)

portf_vol = []
for i in range(0, len(weights)):
  vol = np.sqrt(
      np.dot(weights[i].T, np.dot(cov_matrix, weights[i]))
  )
  portf_vol.append(vol)

portf_vol - np.array(portf_vol)

portf_sharpe = portf_returns / portf_vol

In [ ]:
portf_results = pd.DataFrame({
    'Returns': portf_returns,
    'Volatility': portf_vol,
    'Sharpe Ratio': portf_sharpe
})

In [ ]:
n_points = 100

ef_return_list = []
ef_vol_list = []

possible_ef_return = np.linspace(
    portf_results['Returns'].min(),
    portf_results['Returns'].max(),
    n_points)

possible_ef_return = np.round(possible_ef_return, 2)
portf_returns = np.round(portf_returns, 2)

for rtn in possible_ef_return:
  if rtn in portf_returns:
    ef_return_list.append(rtn)
    matched_ind = np.where(portf_returns == rtn)
    ef_vol_list.append(np.min(portf_vol[matched_ind]))

In [ ]:
markers = ['o','x','d','*']

fig, ax = plt.subplots()
portf_results_df.plot(kind='scatter',x='volatility',y='returns',c='sharpe_ratio',
                      cmap='RdYlGn', edgecolors='black', ax=ax)
ax.set(xlabel='Volatility', ylabel='Expected Returns', title='Efficient Frontier')
ax.plot(ef_vol_list, ef_return_list, 'b--')
for asset_index in range(n_assets):
  ax.scatter(x=np.sqrt(cov_matrix.iloc[asset_index, asset_index]),
             y=avg_returns[asset_index],
             marker=markers[asset_index],
             s=150, color='black', label=assets[asset_index])
ax.legend()
plt.show()

In [ ]:
max_sharpe_ind = np.argmax(portf_results['Sharpe Ratio'])
max_sharpe_portf = portf_results.iloc[max_sharpe_ind]

min_vol_ind = np.argmin(portf_results['Volatility'])
min_vol_portf = portf_results.iloc[min_vol_ind]

In [ ]:
fig, ax = plt.subplots()
portf_results_df.plot(kind='scatter',x='volatility',y='returns',c='sharpe_ratio',
                      cmap='RdYlGn', edgecolors='black', ax=ax)
ax.scatter(x=max_sharpe_port['volatility'],y=max_sharpe_portf['returns'],
           c='black', marker='*',s=200,label='Max Sharpe')
ax.scatter(x=min_vol_portf['volatility'],y=min_vol_portf['returns'],
           c='black', marker='x',s=200,label='Min Volatility')
ax.set(xlabel='Volatility', ylabel='Expected Returns', title='Efficient Frontier')
ax.legend()
plt.show()